In [88]:
print('nyan!')

nyan!


In [89]:
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime

# BIG6 + α
BIG6_CODE = {
    'アーセナル',
    'マンチェスター・U',
    'マンチェスター・ユナイテッド',
    'マンチェスター・C',
    'マンチェスター・シティー',
    'トッテナム',
    'トッテナム・ホットスパー',
    'チェルシー',
    'リヴァプール',
    'ニューカッスル',
    'ニューカッスル・ユナイテッド',
    'リーズ',
    'リーズ・ユナイテッド',
    'ブレントフォード'
}

# 曜日（英語 ⇒ 日本語）
WEEK_DAYS_DICT: dict = MappingProxyType({
        'Mon':'（月）',
        'Tue':'（火）',
        'Wed':'（水）',
        'Thu':'（木）',
        'Fri':'（金）',
        'Sat':'（土）',
        'Sun':'（日）'
})

In [90]:
def scrapeSportsNaviPage(t_yyyymmdd: str, t_en_week_name: str, y_yyyymmdd: str, y_en_week_name: str):
    """
    SportsNaviより引数で指定された開催日・曜日（英語）より、対戦相手および試合日時を取得する関数。
    """
    print('scrapeSportsNaviPage start')
    print('引数⇒', t_yyyymmdd, t_yyyymmdd_week, y_yyyymmdd, y_yyyymmdd_week)

    # 曜日の英語 ⇒ 日本語へ変換
    t_ja_week_name = WEEK_DAYS_DICT.get(t_en_week_name)
    y_ja_week_name = WEEK_DAYS_DICT.get(y_en_week_name)
    
    # 検索キーの作成
    t_search_key = str(int(t_yyyymmdd[4:6])) + '/' + str(int(t_yyyymmdd[6:8])) + t_ja_week_name
    y_search_key = str(int(y_yyyymmdd[4:6])) + '/' + str(int(y_yyyymmdd[6:8])) + y_ja_week_name
    print(t_search_key, y_search_key)

    # 対象ページ
    url = 'https://soccer.yahoo.co.jp/ws/category/eng'

    # スクレイピング
    top_r = requests.get(url)

    # HTMLを解析する
    top_soup = BeautifulSoup(top_r.content, "html.parser")

    # テーブル情報を取得
    td_list = top_soup.find\
            ('table', class_='sc-tableGame').find_all('td')

    # 日付
    match_datetime_list_result = []
    match_datetime_list = []

    # ホームチーム
    home_team_name_list = []

    # アウェーチーム
    away_team_name_list = []    
    
    for td in td_list:
            # 検索対象の開催日の場合（00:00～24:00）
            if t_search_key in td.text:
                work_td = td.text.split('\n')
                # print(work_td)
                for name in work_td:
                    # MMDD
                    # print(name)
                    if re.search(r'(\d*):(\d*)', name) is not None:
                        match_datetime_list_result.append(name)
    for td in td_list:
            # 検索対象の開催日の場合（24:01～）
            if y_search_key in td.text:
                work_td = td.text.split('\n')
                # print(work_td)
                for name in work_td:
                    # MMDD
                    # print(name)
                    if re.search(r'(\d*):(\d*)', name) is not None:
                        match_datetime_list_result.append(name)

    print('scrapeSportsNaviPage end')
    
    for name in match_datetime_list_result:
        #print(name)
        name = name.lstrip()
        #print(name)
        match_datetime_list.append(name)

        
    return match_datetime_list;

In [91]:
# 現在時刻の取得
today = datetime.datetime.now() 
t_yyyymmdd = today.strftime("%Y%m%d")
t_yyyymmdd_week = today.strftime("%a")

print(nowaday, t_yyyymmdd, t_yyyymmdd_week)

# 操作後の日付取得
yesterday = nowaday + datetime.timedelta(-1)
y_yyyymmdd = yesterday.strftime("%Y%m%d")
y_yyyymmdd_week = yesterday.strftime("%a")

print(tomorrow, y_yyyymmdd, y_yyyymmdd_week)

2023-01-04 05:17:43.376994 20230104 Wed
2023-01-03 05:13:51.593117 20230103 Tue


In [92]:
# 関数呼び出し
match_info_list = []
match_info_list = scrapeSportsNaviPage(t_yyyymmdd[0:8], t_yyyymmdd_week, y_yyyymmdd[0:8], y_yyyymmdd_week)
print(match_info_list)

scrapeSportsNaviPage start
引数⇒ 20230104 Wed 20230103 Tue
1/4（水） 1/3（火）
scrapeSportsNaviPage end
['28:45', '28:45', '28:45', '29:00']
